# import library

In [26]:
import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import f1_score, classification_report
import numpy as np
import pandas as pd
import os

# 모델 정의

In [27]:
# Dataset class
class MultiLabelDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        
        labels = [label + 1 for label in labels]
        encoding['labels'] = torch.tensor(labels, dtype=torch.long)
        return encoding

# Model class
class CustomRobertaForMultiLabelClassification(nn.Module):
    def __init__(self, num_labels_per_category=3, num_categories=5):
        super(CustomRobertaForMultiLabelClassification, self).__init__()
        self.roberta = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=num_labels_per_category)
        self.num_categories = num_categories
        self.classifiers = nn.ModuleList([nn.Linear(self.roberta.config.hidden_size, num_labels_per_category) for _ in range(num_categories)])

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.roberta.roberta(input_ids=input_ids, attention_mask=attention_mask)
        logits = torch.stack([classifier(outputs.last_hidden_state[:, 0, :]) for classifier in self.classifiers], dim=1)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            losses = [loss_fct(logits[:, i, :], labels[:, i]) for i in range(self.num_categories)]
            loss = sum(losses) / self.num_categories
        
        return {"loss": loss, "logits": logits}


# 함수 정의

## def train

In [28]:
# Training function
def train(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    
    progress_bar = tqdm(dataloader, desc="Training", leave=False)
    for batch in progress_bar:
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].squeeze(1).to(device)
        attention_mask = batch['attention_mask'].squeeze(1).to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs['loss']
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())
    
    avg_loss = total_loss / len(dataloader)
    return avg_loss



## def prediction

In [30]:
# Prediction function
def predict(model, sentence, tokenizer, device):
    model.eval()
    
    inputs = tokenizer(
        sentence,
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs['logits']
        predictions = torch.argmax(logits, dim=-1).cpu().numpy().squeeze() - 1
    
    labels = ['Product_Quality', 'Delivery', 'Price', 'Repurchase_Intention', 'etc']
    prediction_dict = {label: pred for label, pred in zip(labels, predictions)}
    

    return prediction_dict

# Data Load

In [31]:
data = pd.read_csv('../data/comment_translation_labeling.csv')
data.head()

,review_comment_message,review_comment_message_en,Product_Quality,Delivery,Price,Repurchase_Intention,etc
0,"Produto preto, reembalado na caixa de um branc...","Black product, repackaged in a white box. Very...",0.0,-1.0,0.0,0.0,0.0
1,FICAMOS ESPERANDO UMA ENCOMENDA PAGA QUE NUNCA...,WE ARE WAITING FOR A PAID ORDER THAT NEVER ARR...,0.0,-1.0,0.0,0.0,-1.0
2,Otimo muito bom,Excellent very good,0.0,0.0,0.0,0.0,1.0
3,Entrega antes do prazo. Produto muito bom e bo...,Delivery ahead of schedule. Very good and beau...,1.0,1.0,0.0,0.0,0.0
4,excelente.,excellent.,0.0,0.0,0.0,0.0,1.0


## 라벨링 되어 있는 data 추출

In [45]:
df = data.iloc[:1100,:]
df.tail()

,review_comment_message,review_comment_message_en,Product_Quality,Delivery,Price,Repurchase_Intention,etc
1095,muito bom!,very good!,0.0,0.0,0.0,0.0,1.0
1096,"Produto comprado foi Moringa em cápsulas, a qu...","The product purchased was Moringa in capsules,...",-1.0,0.0,0.0,0.0,0.0
1097,Loja nota 10,Store note 10,0.0,0.0,0.0,0.0,1.0
1098,M/ referência se dará só qto a compra e entreg...,The reference will only be given when the prod...,0.0,1.0,0.0,0.0,0.0
1099,Muito bom chegou antes do previsto,Very good arrived earlier than expected,0.0,1.0,0.0,0.0,1.0


## text와 label 리스트 추출

In [46]:
texts = df['review_comment_message_en'].tolist()
labels = df[['Product_Quality', 'Delivery', 'Price', 'Repurchase_Intention', 'etc']].values.tolist()

print("Texts:", texts)
print("Labels:", labels)

Texts: ["Black product, repackaged in a white box. Very dented at the bottom in a way that no one would notice, it doesn't stabilize. Box in perfect condition, was crushed before being delivered to the post office.", 'WE ARE WAITING FOR A PAID ORDER THAT NEVER ARRIVED AND THE PURCHASE VALUE HAS STILL NOT BEEN REFUNDED', 'Excellent very good', 'Delivery ahead of schedule. Very good and beautiful product.', 'excellent.', 'Delivery came in half', "I ordered two tubs and only received one. I'm waiting for an answer on how I can resolve this, but the delivery was on time and the one I received is of good quality.", 'Product delivered in perfect condition and well before the stipulated delivery date', 'Great product', "Good evening, I received the goods partially because a tub is broken, what should I do? I'm at a loss or who is going to replace this broken situation?", 'Ameiiii!!! excellent product...excellent quality, delivered very well ahead of schedule, thank you', 'In economic shipping

# 토크나이저 로드 및 처리

In [47]:
# # Sample data
# texts = ["Black product, repackaged in a white box...", "WE ARE WAITING FOR A PAID ORDER..."]
# labels = [[0, -1, 0, 0, 0], [0, -1, 0, 0, -1]]
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
dataset = MultiLabelDataset(texts, labels, tokenizer)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)

# 모델 초기화

In [48]:
model = CustomRobertaForMultiLabelClassification()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Python310\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# train

In [49]:
num_epochs = 5
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    avg_loss = train(model, train_loader, optimizer, device)
    print(f"Training Loss: {avg_loss}")

Epoch 1/5


Training:   0%|          | 0/138 [00:00<?, ?it/s]

Training Loss: 0.515200050736683
Epoch 2/5


Training Loss: 0.27009070891401044
Epoch 3/5


Training Loss: 0.1852645972824615
Epoch 4/5


Training Loss: 0.1337185048268757
Epoch 5/5


Training Loss: 0.10150474597416494


# 모델 저장

In [50]:
# 모델 가중치를 저장할 경로 지정
model_save_path = "../model/custom_roberta_model_1100_5epoch.pth"

# 모델 가중치 저장
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to ../model/custom_roberta_model_1100_5epoch.pth


# 파인튜닝 모델 LOAD

## model load 함수

In [41]:
def load_model(model_path, device = torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    model = CustomRobertaForMultiLabelClassification()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()  # 평가 모드로 설정
    print(f"Model loaded from {model_path}")
    return model

# f1 score

## 함수

전체 나열, 라벨별, 리뷰별 순으로 f1 score 리턴

In [14]:
# 3가지 F1-Score 계산 함수 정의
def calculate_f1_scores(test_labels, test_predictions):
    # 전체 레이블을 평탄화하여 F1-Score 계산
    flattened_true_labels = [label for sublist in test_labels for label in sublist]
    flattened_predictions = [pred for sublist in test_predictions for pred in sublist]
    overall_f1 = f1_score(flattened_true_labels, flattened_predictions, average="micro")

    # 각 레이블별 F1-Score 계산
    label_names = ['Product_Quality', 'Delivery', 'Price', 'Repurchase_Intention', 'etc']
    label_f1_scores = {}
    for i, label_name in enumerate(label_names):
        true_labels = [label[i] for label in test_labels]
        pred_labels = [pred[i] for pred in test_predictions]
        label_f1 = f1_score(true_labels, pred_labels, average="micro")
        label_f1_scores[label_name] = label_f1

    # 리뷰 단위 정확한 매칭 기반 F1-Score 계산
    review_matches = [1 if true == pred else 0 for true, pred in zip(test_labels, test_predictions)]
    true_review_matches = [1] * len(review_matches)
    review_level_f1 = f1_score(true_review_matches, review_matches, average="binary")

    return overall_f1, label_f1_scores, review_level_f1

## test data load

In [54]:
test_df = data.iloc[1100:1200, :]
test_df.tail()


,review_comment_message,review_comment_message_en,Product_Quality,Delivery,Price,Repurchase_Intention,etc
1195,"Maravilhoso, e a entrega super rapido, recomendo.","Wonderful, and super fast delivery, I recommen...",0.0,1.0,0.0,1.0,0.0
1196,chegou tudo certo,everything arrived ok,0.0,0.0,0.0,0.0,1.0
1197,parece bom ainda não usei.,"It looks good, I haven't used it yet.",1.0,0.0,0.0,0.0,0.0
1198,"não recebi o produto completo como deveria, co...",I didn't receive the complete product as I sho...,0.0,0.0,0.0,0.0,1.0
1199,ótimo produto.\nchegou bem rápido .\nem menos ...,great product.\narrived very quickly.\nin less...,1.0,1.0,0.0,0.0,0.0


In [55]:
# 리뷰 텍스트와 레이블 추출
test_texts = test_df['review_comment_message_en'].tolist()
test_labels = test_df[['Product_Quality', 'Delivery', 'Price', 'Repurchase_Intention', 'etc']].values.tolist()


## 단일 모델 f1-score

### 모델 불러오기

In [56]:
model_path = "../model/custom_roberta_model_1100_5epoch.pth"

In [57]:
loaded_model = load_model(model_path)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_7744\2575577657.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via t

Model loaded from ../model/custom_roberta_model_1100_5epoch.pth


### f1 score 출력(단일 모델)

In [58]:
# 예측 수행
test_predictions = []
for sentence in test_texts:
    prediction = predict(loaded_model, sentence, tokenizer, device)
    test_predictions.append([
        prediction['Product_Quality'],
        prediction['Delivery'],
        prediction['Price'],
        prediction['Repurchase_Intention'],
        prediction['etc']
    ])

# 3. F1-Score 계산
overall_f1, label_f1_scores, review_level_f1 = calculate_f1_scores(test_labels, test_predictions)

# 결과 출력
print(f"Results for model {model_path}:")
print(f"  Overall F1-Score: {overall_f1:.4f}")
for label_name, f1 in label_f1_scores.items():
    print(f"  {label_name} F1-Score: {f1:.4f}")
print(f"  Review-level Exact Match F1-Score: {review_level_f1:.4f}")
print("--------------------------------------------------")

Results for model ../model/custom_roberta_model_1100_5epoch.pth:
  Overall F1-Score: 0.9100
  Product_Quality F1-Score: 0.9000
  Delivery F1-Score: 0.9000
  Price F1-Score: 0.9900
  Repurchase_Intention F1-Score: 0.9800
  etc F1-Score: 0.7800
  Review-level Exact Match F1-Score: 0.7879
--------------------------------------------------


## 모든 모델 f1-score

### 모델 리스트 불러오기

In [40]:
model_folder = "../model"
model_files = [f for f in os.listdir(model_folder) if f.endswith(".pth")]


### f1 score 출력 (모든 모델)

In [25]:
for model_file in tqdm(model_files):
    # 모델 불러오기
    model_path = os.path.join(model_folder, model_file)
    model = CustomRobertaForMultiLabelClassification()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    print(f"\nLoaded model: {model_file}")

    # 예측 수행
    test_predictions = []
    for sentence in test_texts:
        prediction = predict(model, sentence, tokenizer, device)
        test_predictions.append([
            prediction['Product_Quality'],
            prediction['Delivery'],
            prediction['Price'],
            prediction['Repurchase_Intention'],
            prediction['etc']
        ])

    # 3. F1-Score 계산
    overall_f1, label_f1_scores, review_level_f1 = calculate_f1_scores(test_labels, test_predictions)

    # 결과 출력
    print(f"Results for model {model_file}:")
    print(f"  Overall F1-Score: {overall_f1:.4f}")
    for label_name, f1 in label_f1_scores.items():
        print(f"  {label_name} F1-Score: {f1:.4f}")
    print(f"  Review-level Exact Match F1-Score: {review_level_f1:.4f}")
    print("--------------------------------------------------")

  0%|          | 0/5 [00:00<?, ?it/s]Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_7744\3666518439.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no


Loaded model: custom_roberta_model_1000_3epoch.pth


 20%|██        | 1/5 [01:31<06:06, 91.61s/it]

Results for model custom_roberta_model_1000_3epoch.pth:
  Overall F1-Score: 0.9060
  Product_Quality F1-Score: 0.8500
  Delivery F1-Score: 0.8950
  Price F1-Score: 0.9800
  Repurchase_Intention F1-Score: 0.9700
  etc F1-Score: 0.8350
  Review-level Exact Match F1-Score: 0.7879
--------------------------------------------------


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_7744\3666518439.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via t

...

Results for model custom_roberta_model_800.pth:
  Overall F1-Score: 0.8980
  Product_Quality F1-Score: 0.8550
  Delivery F1-Score: 0.9000
  Price F1-Score: 0.9850
  Repurchase_Intention F1-Score: 0.9700
  etc F1-Score: 0.7800
  Review-level Exact Match F1-Score: 0.7692
--------------------------------------------------


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_7744\3666518439.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via t


Loaded model: custom_roberta_model_900.pth


100%|██████████| 5/5 [07:09<00:00, 85.93s/it]

Results for model custom_roberta_model_900.pth:
  Overall F1-Score: 0.8870
  Product_Quality F1-Score: 0.8800
  Delivery F1-Score: 0.8950
  Price F1-Score: 0.9800
  Repurchase_Intention F1-Score: 0.9400
  etc F1-Score: 0.7400
  Review-level Exact Match F1-Score: 0.7578
--------------------------------------------------


# 모델 성능 평가

1. Results for model custom_roberta_model_600.pth:   
  - Overall F1-Score: 0.8980    
  - Product_Quality F1-Score: 0.8750    
  - Delivery F1-Score: 0.9000   
  - Price F1-Score: 0.9800    
  - Repurchase_Intention F1-Score: 0.9550   
  - etc F1-Score: 0.7800    
  - Review-level Exact Match F1-Score: 0.7578   
--------------------------------------------------

2. Results for model custom_roberta_model_700.pth:
  - Overall F1-Score: 0.8990
  - Product_Quality F1-Score: 0.8650
  - Delivery F1-Score: 0.8800
  - Price F1-Score: 0.9800
  - Repurchase_Intention F1-Score: 0.9550
  - etc F1-Score: 0.8150
  - Review-level Exact Match F1-Score: 0.7768
--------------------------------------------------

3. Results for model custom_roberta_model_800.pth:
  - Overall F1-Score: 0.8980
  - Product_Quality F1-Score: 0.8550
  - Delivery F1-Score: 0.9000
  - Price F1-Score: 0.9850
  - Repurchase_Intention F1-Score: 0.9700
  - etc F1-Score: 0.7800
  - Review-level Exact Match F1-Score: 0.7692
--------------------------------------------------

4. Results for model custom_roberta_model_900.pth:
  - Overall F1-Score: 0.8870
  - Product_Quality F1-Score: 0.8800
  - Delivery F1-Score: 0.8950
  - Price F1-Score: 0.9800
  - Repurchase_Intention F1-Score: 0.9400
  - etc F1-Score: 0.7400
  - Review-level Exact Match F1-Score: 0.7578
--------------------------------------------------

5. <b>Results for model custom_roberta_model_1000_3epoch.pth</b>:
  - Overall F1-Score: 0.9060
  - Product_Quality F1-Score: 0.8500
  - Delivery F1-Score: 0.8950
  - Price F1-Score: 0.9800
  - Repurchase_Intention F1-Score: 0.9700
  - <u>etc F1-Score: 0.8350</u>
  - Review-level Exact Match F1-Score: 0.7879
--------------------------------------------------

6. Results for model ../model/custom_roberta_model_1000_5epoch.pth:
  - Overall F1-Score: 0.9070
  - Product_Quality F1-Score: 0.9000
  - Delivery F1-Score: 0.9250
  - Price F1-Score: 0.9850
  - Repurchase_Intention F1-Score: 0.9500
  - etc F1-Score: 0.7750
  - Review-level Exact Match F1-Score: 0.7842
--------------------------------------------------

7. Results for model ../model/custom_roberta_model_1100_5epoch.pth:
  - Overall F1-Score: 0.9100
  - Product_Quality F1-Score: 0.9000
  - Delivery F1-Score: 0.9000
  - Price F1-Score: 0.9900
  - Repurchase_Intention F1-Score: 0.9800
  - etc F1-Score: 0.7800
  - Review-level Exact Match F1-Score: 0.7879
--------------------------------------------------

